In [1]:
import pyslha #For interacting with SLHA files
import subprocess #Running command
import scipy #Numeric Library
import sympy #symbolic computation - might not be used here
from scipy import optimize
from sympy import *
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import numpy as np
%matplotlib inline
from numpy import logspace, linspace 
from itertools import product
import glob #filehandling
from subprocess import call
import os
#Path to modified verison of Softsusy 
SoftSUSY_PATH = 'softsusy-4.1.4-Garon/softpoint.x' 
mHiggs = 125 #Higgs Mass for paper
print ("hello")

hello


In [7]:
def slhaPoint(model='FGMCaseB0', mMess=1e12, tanBeta=10, beta2=0, beta3=0, betaeps=.01, thetavev=0, 
              outputFile='test.slha', Lambda=1e6, mode='Fit', LambdaGuess=6e5,particle=25):
    '''
    Code for interacting with SoftSUSY and running the models we've been looking at.
    The models themselves are encoded in a modified version of it. 
    This may seem like extra work because it accepts an arbitrary SLHA2 file, but since our models
    include fitted parameters, like the Yukawa couplings, we need to use the values that it calculates
    during the running. 
    The parameters:
    model - the name of the model in SoftSUSY
    mMess - the messenger scale
    tanBeta - tan beta
    beta2 - beta2
    beta3 - beta3
    outputFile - the name of the file to write to (this step could conceivably be removed by writing the console output to memory)
    Lambda - F/M
    mode - There are 3 modes
        'Fit' - uses Newton's method from a few different starting points
        'Bisect' - uses a Bisection routine from a few different brackets
        'Point' - runs a single point
    LambdaGuess - a guess at the value of Lambda for the 'Fit' mode.
    particle - the pdg code for the particle that we are interested in getting data for
    
    note that current implementation, fit and bisect only work for higgs, could fix, but it could only really work for
    particles with known values
    '''
    
    dataline ="./{} {} --mMess={} --LAMBDA={} --tanBeta={} --beta2={} --beta3={} --thetavev={} --betaeps={} --cgrav=1. > {}"
    startingPoints = [LambdaGuess, 1e6, 5e6, 6e4]
    bracket, l = [(2,5), (5,10), (2,10), (2,8), (4,8), (1,3), (8,12)], 0
    
    #a helper functin that takes a value of x (Lambda), runs it (line 2) and then reads in the Higgs mass
    def f(x):
        string = dataline.format(SoftSUSY_PATH, model, str(mMess), str(x), 
                                 str(tanBeta), str(beta2), str(beta3), str(thetavev), str(betaeps), outputFile)
        subprocess.Popen(string, stdout=None, shell=True).wait()
        #print ("the string is ")
        #print (string)
        return pyslha.read(outputFile).blocks['MASS'][particle]
    
    if mode == 'Fit':
        #print ('in fit')
        a, i = 0, 0
        while((a <= 0) & (i < len(startingPoints))):
            try:
                #Newton's method
                print ("in try")
                a = optimize.newton(f, startingPoints[i], tol=1e-4)
                print ("in newton")
                print (a)
            except:
                print ('got excepted')
                a = 0
                i += 1
        if (a>0):
            data = pyslha.read(outputFile)
            #accept or reject based on the deviation from the Higgs mass
            if (-0.1 < data.blocks['MASS'][25]-125 < 0.1) :
                try:
                    #if there's something in this location SoftSUSY has an error.
                    data.blocks['SPINFO'][3]
                except:
                    return data
    elif mode == 'Bisect':
        for k in bracket:
            print(k)
            try:
                l = optimize.brentq(f, k[0] * 1e5, k[1] * 1e5, rtol=1e-4)
                if l:
                    break
            except:
                #print('nada')
                continue
        if l:
            data = pyslha.read(outputFile)
            print(data.blocks['MASS'][25])
            if (-0.25 < data.blocks['MASS'][25]-125 < 0.25) :
                try:
                    data.blocks['SPINFO'][3]
                except:
                    return data
            else:
                #the bisection routine tends to get things close but not exact, so it calls 'Fit' if it's close.
                return slhaPoint(model, mMess, tanBeta, beta2, beta3,\
                              outputFile, Lambda, 'Fit', LambdaGuess=data.blocks['MINPAR'][1])
        else:
            #just to be safe (but is probably unneded)
            return slhaPoint(model, mMess, tanBeta, beta2, beta3,\
                              outputFile, Lambda, 'Fit')
    else:
        #runs a single point
        try:
            f(Lambda)
            data = pyslha.read(outputFile)
            return data.blocks['MASS'][particle], data
        except (pyslha.ParseError, RuntimeError):
            return "No Solution", 0

In [8]:
gd = slhaPoint(model='FGMCaseB1',outputFile="test3.slha",particle=2000015,beta2=10,beta3=10)

in try


Ignoring unknown section type: in
Ignoring unknown section type: in
Ignoring unknown section type: in


got excepted
in try


Ignoring unknown section type: in
Ignoring unknown section type: in
Ignoring unknown section type: in


got excepted
in try


Ignoring unknown section type: in
Ignoring unknown section type: in
Ignoring unknown section type: in


got excepted
in try
got excepted


Ignoring unknown section type: in
Ignoring unknown section type: in


In [11]:
gd

In [ ]:
gridBeta2 = logspace(0.0,2.0,41)
gridBeta3 = logspace(0.0,2.0,41)

resultsFGMCaseB1 = {}
k = 0
for j in product(gridBeta2, gridBeta3):
    k+=1
    print(k)
    resultsFGMCaseB1[j] = slhaPoint(model='FGMCaseB1',beta2= j[0], beta3= j[1])
print("done")    

1
in try


Ignoring unknown section type: in
Ignoring unknown section type: in
Ignoring unknown section type: in


got excepted
in try


Ignoring unknown section type: in
Ignoring unknown section type: in
Ignoring unknown section type: in


got excepted
in try


...

got excepted
in try


Ignoring unknown section type: in
Ignoring unknown section type: in
Ignoring unknown section type: in


got excepted
1538
in try


Ignoring unknown section type: in
Ignoring unknown section type: in
Ignoring unknown section type: in


got excepted
in try


In [ ]:
file = resultsFGMCaseB1
plotData, plotData2, plotData3 = [], [], []
for i in gridBeta2:
    row, row2, row3 = [], [], []
    for j in gridBeta3:
        try:
            row.append(file[(j,i)][1].blocks['MASS'][25])
            row2.append(file[(j,i)][1].blocks['MASS'][2000002])
            row3.append(file[(j,i)][1].blocks['MASS'][2000011])
            #row.append(resultsB1mMessLambda40[(j,i)][1].blocks['MINPAR'][1])
        except:
            row.append(0)
            row2.append(0)
            row3.append(0)
            #print((j,i))
    plotData.append(row)
    plotData2.append(row2)
    plotData3.append(row3)
plotData = np.array(plotData)
plotData2 = np.array(plotData2)
plotData3 = np.array(plotData3)
print("plotData is")
#print(plotData)

Zm = np.ma.masked_where(plotData < 1.2, plotData)
Zn = np.ma.masked_where(plotData2 < 1.2, plotData2)
Zo = np.ma.masked_where(plotData3 < 1.2, plotData3)
print("zm, then zn, then z0")
print(Zm)
print("zn")
print(Zn)
print('z0')
print(Zo)

fig, ax = plt.subplots( figsize=(8, 6.4))

im = ax.contourf(np.log10(gridBeta3), np.log10(gridBeta2), Zo,50)
plt.xlabel(r'$\log_{10}\, M_{\rm{Beta3}}$',fontsize=16)
plt.ylabel(r'$\log_{10}\,M_{\rm{Beta2}}$',fontsize=16)
cbar = fig.colorbar(im)
cbar.set_label(r'$m_{\tilde C_{R}}$ [GeV]', size=12)
cs = ax.contour(np.log10(gridBeta3), np.log10(gridBeta2), Zm, [124,125,126,127,128],colors='k',linestyles='dotted')
ax.clabel(cs, fontsize=16, inline=1, colors='w',fmt="%1.2f")
#cs = ax.contour(np.log10(gridBeta3), np.log10(gridBeta2), Zn, [12000,12100,12200,12300],colors='k',linestyles='dotted')
#ax.clabel(cs, fontsize=16, inline=1, colors='w',fmt="%1.1f")
#cs = ax.contour(np.log10(gridBeta3), np.log10(gridBeta2), Zo, [4600],colors='k',linestyles='dotted')
#ax.clabel(cs, fontsize=16, inline=1, colors='w',fmt="%1.f")
plt.title("2000011 Mass Spectrum")
plt.savefig("b3b2eR.pdf")
